In [1]:
import joblib
import numpy as np

from langchain.chat_models import ChatOpenAI
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent

import dotenv
import os

dotenv.load_dotenv("../../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
hotpot = joblib.load('hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

hotpot['supporting_paragraphs'] = None
for ind, row in hotpot.iterrows():
    supporting_articles = row['supporting_facts']['title']
    articles = row['context']['title']
    sentences = row['context']['sentences'] 
    supporting_paragraphs = []
    for article in supporting_articles:
        supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
        supporting_paragraphs.append(supporting_paragraph)
    supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
    hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs

In [3]:
row = hotpot.iloc[0]

llm = ChatOpenAI(
    temperature=0,
    max_tokens=250,
    model_name="gpt-3.5-turbo",
    model_kwargs={"stop": "\n"},
    openai_api_key=openai_api_key
)

agent = ReflexionReActAgent(
    self_reflect_llm=llm,
    action_llm=llm,
)

In [4]:
row["question"], row["answer"], row["supporting_paragraphs"]

("VIVA Media AG changed it's name in 2004. What does their new acronym stand for?",
 'Gesellschaft mit beschränkter Haftung',
 'VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. It was founded for broadcast of VIVA Germany as VIVA Media AG in 1993 and has been owned by their original concurrent Viacom, the parent company of MTV, since 2004. Viva channels exist in some European countries; the first spin-offs were launched in Poland and Switzerland in 2000.\n\nA Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. In the United States, the equivalent type of entity is the limited liability company (LLC). The name of the GmbH form emphasizes the fact that the owners ("Gesellschafter", also known as members) of the entity are not personally liable for the company\'s debts. "Gm

In [5]:
out = agent.generate(
    question=row["question"], 
    key=row["answer"], 
    strategy=None
)
print("================================================", "FINAL OUTPUT", "================================================")
print(out)

THOUGHT
PROMPT START ================================================================================================

Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You may take as many steps as necessary.
Here are some examples:

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Sear

In [6]:
out = agent.generate(
    question=row["question"], 
    key=row["answer"], 
    strategy="last_attempt_and_reflexion"
)
print("================================================", "FINAL OUTPUT", "================================================")
print(out)

REFLECT PROMPT START ================================================================================================

You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to an Docstore API environment and a question to answer. You were unsuccessful in answering the question either because you guessed the wrong answer with Finish[<answer>], or you used up your set number of reasoning steps. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences.  
Here are some examples:

Previous Trial:
Question: The Rome Protocols were signed by three Prime Ministers one of which was assassinated as part of what?
Thought 1: I need to search Rome Protocols, find the three Prime Ministers, then find what they were assassinated as part of.
Action 1: Search[Rome Protocols]
Observation 1: The Rome